# install & import

In [ ]:
!pip install transformers datasets peft accelerate

In [2]:
import re
import csv
import random
import torch
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from datasets import Dataset, DatasetDict


from transformers import (AutoTokenizer,
                          BertForTokenClassification,
                          DataCollatorForTokenClassification,
                          TrainingArguments, Trainer,
                          AutoModelForSequenceClassification)


from sklearn.metrics import f1_score, accuracy_score

from peft import LoraConfig, get_peft_model, TaskType, PeftModel


# Подготовка датасета

## Обработка первого датасета с литературой

In [20]:
def clean_text(text):
    text = re.sub(r"—", " ", text)      # тире-роль на пробел
    text = text.replace("\x97", " ").replace("\xad", " ")
    text = re.sub(r"\s+", " ", text)    # все пробельные символы (\n, \t, множественные пробелы) -> один пробел
    return text.strip()

def split_into_fragments(text, words_per_fragment=10):
    """
    Разбиваем текст на фрагменты по N слов.
    """
    words = text.split()
    fragments = []

    for i in tqdm(range(0, len(words), words_per_fragment), desc="Processing words"):
        fragment = " ".join(words[i:i+words_per_fragment])
        fragments.append(fragment)

    return fragments

In [5]:
path = '/content/drive/MyDrive/Контесты/2025_avito_internship/'
# path = '/content/drive/MyDrive/'

with open(path + 'books-B.txt', "r", encoding="utf-8") as f:
    text = f.read()

text = clean_text(text)

# Разделение на фрагменты
fragments = split_into_fragments(text)

Processing words:   0%|          | 0/310326 [00:00<?, ?it/s]

In [6]:
fragments[:5]

['В смысле они улетели? с неимоверным удивлением спрашиваю у Арти,',
 'все еще не в силах поверить в это, слишком уж',
 'неожиданно и вообще странно как-то. В прямом, флот вышел в',
 'звездной системе недалеко от нас, предполагаю для корректировки курса, они',
 'так делают при прыжках на очень большие расстояния, а потом']

## Обработка 2-го датасета с объявлениями


    df = pd.read_parquet(path + 'train_part_0002.snappy.parquet')
    df.to_csv('avito2.csv')

    newdf = df[['cand_description']]

    def filter_text(text):
        if pd.isna(text):
            return ""
        # оставляем только буквы, цифры, пробелы и знаки препинания
        # \w = буквы и цифры, \s = пробелы, а .,!?;: оставляем знаки препинания
        text = re.sub(r"[^()\w\s.,!?;:]", "", text)
        return text


    # Функция для получения первых 12 слов
    def first_12_words(text):
        if pd.isna(text):
            return ""
        text = filter_text(text)
        words = text.split()
        return " ".join(words[:12])

    # Применяем ко всему столбцу
    newdf['first12'] = newdf['cand_description'].apply(first_12_words)

    savedf = newdf.iloc[:, [0, -1]]
    savedf = savedf.drop_duplicates(subset=['first12'], keep='first', ignore_index=True)
    savedf.to_csv(path + 'avito_dataset.csv')

In [7]:
df_avito = pd.read_csv(path + 'avito_dataset.csv', usecols = [1, 2])
df_avito.head(3)

,cand_description,first12
0,"Воpотник пеcец , pукaвa и низ дубленки кpолик ...","Воpотник пеcец , pукaвa и низ дубленки кpолик ..."
1,"Доcтaвкой не отпpaвляю ! Новые ,ни paзу не оде...","Доcтaвкой не отпpaвляю ! Новые ,ни paзу не оде..."
2,"Кpоccовки Demix, paзмеp 34 (длинa cтопы 22 cм)","Кpоccовки Demix, paзмеp 34 (длинa cтопы 22 cм)"


## Сбор полноценного датасета

In [21]:
len1, len2 = len(fragments), df_avito.shape[0]
dataset = fragments[:] + df_avito.first12.values.tolist()[:]
len(dataset)

623797

In [22]:
def prepare_data(texts):
    data = []
    for text in tqdm(texts):
        text = str(text).strip()
        no_space = ''.join(text.split())  # Удаляем все пробелы
        if len(no_space) < 2:
            continue  # Пропускаем слишком короткие (нет labels)

        labels = [0] * (len(no_space) - 1)  # Инициализируем 0-ми
        char_idx = 0
        for char in text:
            if char == ' ':
                if char_idx > 0:
                    labels[char_idx - 1] = 1  # Пробел после предыдущего символа
            else:
                char_idx += 1

        data.append({'text': no_space, 'labels': [-100] + labels + [1] + [-100]}) #-100 для start end символов потом
    return Dataset.from_list(data)

train_ds = prepare_data(dataset)

  0%|          | 0/623797 [00:00<?, ?it/s]

    print(train_ds[:2])

    
    
    {'text': ['Всмыслеониулетели?снеимовернымудивлениемспрашиваюуАрти,', 'всеещеневсилахповеритьвэто,слишкомуж'], 'labels': [[1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1], [0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1]]}

In [10]:
#просто проверка длины
# for i in tqdm(range(len(train_ds))):
#     if len(train_ds[0]['text']) != len(train_ds[0]['labels']) - 2:
#       print(i, len(train_ds[0]['text']), len(train_ds[0]['labels']) - 2)

## Преобразование в PyTorch Dataset

In [23]:
model_name = 'cointegrated/rubert-tiny2'
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_and_align(examples):
    chars = [list(ex) for ex in examples['text']]
    tokenized = tokenizer(chars, is_split_into_words=True)#, truncation=True, padding='max_length', max_length=128
    tokenized['labels'] = examples['labels']  # Align: pad labels to match seq_len, ignore [CLS]/[SEP]
    return tokenized

tokenized_ds = train_ds.map(tokenize_and_align, batched=True, batch_size = 2000)  #2000 - 3:30  2500 - 4:30

Map:   0%|          | 0/623779 [00:00<?, ? examples/s]

In [ ]:
#Опять проверка размерностей. Были проблемы из-за того, что не удалил "\x97" "\xad"

# for i in tqdm(range(180000, len(tokenized_ds))):
#     if len(tokenized_ds[i]['input_ids']) != len(tokenized_ds[i]['labels']):
#         print(i, len(tokenized_ds[i]['input_ids']), len(tokenized_ds[i]['labels']))


  0%|          | 0/443779 [00:00<?, ?it/s]

In [24]:
#Указываю seed для воспроизводимости
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [25]:
split_ds = tokenized_ds.train_test_split(test_size=0.05, seed=42)

# Чтобы работало с Trainer, лучше собрать в DatasetDict
dataset_torch = DatasetDict({
    "train": split_ds["train"],
    "eval": split_ds["test"]
})
dataset_torch

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 592590
    })
    eval: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 31189
    })
})

# Загрузка и обучение модели

In [26]:
def compute_metrics(pred):
    # pred.predictions — logits [batch_size, seq_len, num_labels]
    # pred.label_ids — реальные метки
    predictions = pred.predictions.argmax(-1)
    labels = pred.label_ids

    # Маскируем паддинги
    mask = labels != -100  #  -100 для игнорируемых токенов
    predictions = predictions[mask]
    labels = labels[mask]

    f1 = f1_score(labels, predictions)
    acc = accuracy_score(labels, predictions)
    return {"f1": f1, "accuracy": acc}

In [27]:
model = BertForTokenClassification.from_pretrained(
    model_name,
    num_labels=2)

data_collator = DataCollatorForTokenClassification(tokenizer)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# lora_config = LoraConfig(
#     r=8,                      # ранг низкоранговой матрицы
#     lora_alpha=32,
#     target_modules=["query", "value"],
#     lora_dropout=0.1,
#     bias="none",
#     task_type=TaskType.TOKEN_CLS
# )

# model = get_peft_model(model, lora_config)

      
    # Проверка, какие параметры обучаемые
    for name, param in model.named_parameters():
        print(f"{name}") if param.requires_grad else ''
  
    base_model.model.bert.encoder.layer.10.attention.self.value.lora_B.default.weight
    base_model.model.bert.encoder.layer.11.attention.self.query.lora_A.default.weight
    base_model.model.bert.encoder.layer.11.attention.self.query.lora_B.default.weight
    base_model.model.bert.encoder.layer.11.attention.self.value.lora_A.default.weight
    base_model.model.bert.encoder.layer.11.attention.self.value.lora_B.default.weight
    base_model.model.classifier.modules_to_save.default.weight
    base_model.model.classifier.modules_to_save.default.bias

In [28]:
args = TrainingArguments(
    output_dir = path + "sber_rubert_space_recovery",
    save_strategy="steps",
    save_steps=1000,
    logging_steps=200,
    learning_rate=5e-5,
    per_device_train_batch_size=32,#32
    per_device_eval_batch_size=32,#32
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    metric_for_best_model="f1",
    eval_strategy="steps",
    eval_steps=1000,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset_torch["train"],
    eval_dataset=dataset_torch["eval"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-3808860287.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [29]:
trainer.train()

Step,Training Loss,Validation Loss,F1,Accuracy
1000,0.149500,0.128342,0.858956,0.953120
2000,0.124900,0.105703,0.885164,0.961700
3000,0.111500,0.095345,0.896995,0.965737
4000,0.103900,0.087556,0.906792,0.968535
5000,0.099200,0.082076,0.912779,0.970503
6000,0.094000,0.077754,0.918296,0.972269
7000,0.089200,0.073982,0.922179,0.973619
8000,0.087700,0.071187,0.925084,0.974559
9000,0.084900,0.068630,0.928402,0.975670
10000,0.081100,0.066541,0.930397,0.976361


TrainOutput(global_step=55557, training_loss=0.07017246627210251, metrics={'train_runtime': 3801.138, 'train_samples_per_second': 467.694, 'train_steps_per_second': 14.616, 'total_flos': 2466666451944648.0, 'train_loss': 0.07017246627210251, 'epoch': 3.0})

    Каждая валидация длилась 15 секунд
    Итого только валидации длились около 15 минут

# Сохранение и загрузка дообученной модели

In [4]:
fold_name = "tiny_rubert_space_recovery"
model_name = 'cointegrated/rubert-tiny2'
# path = '/content/drive/MyDrive/'
path = '/content/drive/MyDrive/Контесты/2025_avito_internship/'

In [5]:
# # сохраняем модель (включая голову token classification)
# trainer.save_model(path + fold_name)

# # сохраняем токенайзер
# tokenizer.save_pretrained(path + fold_name)

In [8]:
#Загрузка модели
tokenizer = AutoTokenizer.from_pretrained(path + fold_name)
fine_tuned_model = BertForTokenClassification.from_pretrained(path + fold_name)

# Проверка работоспособности модели

In [6]:
with open(path + "dataset_1937770_3.txt", "r", encoding="utf-8") as f:
    texts = [line.strip().split(',', 1)[1] for line in f][1:]#- чтобы не считать 'text_no_spaces',
texts[:5]

['куплюайфон14про',
 'ищудомвПодмосковье',
 'сдаюквартирусмебельюитехникой',
 'новыйдивандоставканедорого',
 'отдамдаромкошку']

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fine_tuned_model.to(device)

def recover_spaces(text, model, tokenizer, device):
    chars = list(text)
    inputs = tokenizer(chars, is_split_into_words=True, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    predictions = torch.argmax(outputs.logits, dim=-1).squeeze().tolist()
    predictions = predictions[1:-1]
    res = ''
    for i in range(len(text)) :
        res += text[i]
        if predictions[i] == 1:
            res+= ' '

    return res.strip()


# Проверим первые 5 примеров
for text in texts[100:107]:
    restored = recover_spaces(text, fine_tuned_model, tokenizer, device)
    print(f"\nИсходный: {text}")
    print(f"Восстановленный: {restored}")


Исходный: ищукнигубратьякарамазовы,срочно
Восстановленный: ищу кни гу братья карамазовы, срочно

Исходный: новыймонитор,27дюймов,доставка
Восстановленный: новый монитор, 27 дюймов, доставка

Исходный: куплюковрикдляйоги,недорого!
Восстановленный: куплюковрик для йоги, не дорого!

Исходный: ищуинструкторапоплаванию,бассейнрядом
Восстановленный: ищу инструктора по плаванию, бассейн рядом

Исходный: сдамкомнату,толькодевушке
Восстановленный: с дам комнату, только девушке

Исходный: куплюшвейнуюмашинку,рабочую
Восстановленный: куплю швейную машинку, рабочую

Исходный: ищудрузейдляпутешествий,летомвгоры
Восстановленный: ищу друзей для путешествий, летом в горы


In [10]:
#Модель предсказывает нужен ли пробел ПОСЛЕ символа - надо учесть это
def get_answers(text, model, tokenizer, device):
    chars = list(text)
    inputs = tokenizer(chars, is_split_into_words=True, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    predictions = torch.argmax(outputs.logits, dim=-1).squeeze().tolist()
    predictions = predictions[1:-2] #0, -1 - это для служебных токенов/ -2 - это предсказание нужен ли токен после последнего символа,

    res = [ind + 1 for ind, val in enumerate(predictions) if val == 1]
    return str(res)

#!!! почти всегда предсказывает пробел в конце
val_text = 'книгавхорошемсостоянии'
restored = recover_spaces(val_text, fine_tuned_model, tokenizer, device)
print(f"Восстановленный: {restored}")
get_answers(val_text, fine_tuned_model, tokenizer, device)

Восстановленный: книга в хорошем состоянии


'[5, 6, 13]'

In [11]:
predicted_positions = [get_answers(text, fine_tuned_model, tokenizer, device) for text in tqdm(texts)]

  0%|          | 0/1005 [00:00<?, ?it/s]

In [12]:
res_df = pd.DataFrame({
    'id': range(len(predicted_positions)),
    'predicted_positions': predicted_positions
})
res_df

,id,predicted_positions
0,0,"[5, 10, 12]"
1,1,"[6, 7, 10]"
2,2,"[1, 4, 12, 13, 20, 21]"
3,3,"[5, 10, 18, 20]"
4,4,"[5, 10]"
...,...,...
1000,1000,"[1, 3]"
1001,1001,"[7, 10, 12, 16]"
1002,1002,"[11, 13, 19]"
1003,1003,"[8, 19, 22]"


In [13]:
res_df.to_csv('avito_res5.csv', index=False)